In [3]:
import numpy as np
import pandas as pd
import time
from datetime import timedelta
import networkx as nx
import matplotlib.pyplot as plt
import collections
import random
from network_tolerance_nx import *

In [4]:
unprocesseddf2022 = pd.read_csv('2022-capitalbikeshare-tripdata_py.csv')
unprocesseddf2022.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,1B4445D591115BD9,classic_bike,2022-01-06 18:39:28,2022-01-06 18:43:03,Monroe Ave & Leslie Ave,31087.0,Potomac Ave & Main Line Blvd,31910.0,38.820932,-77.053096,38.822738,-77.049265,member
1,7F4A0E2F03EADEB7,classic_bike,2022-01-31 19:21:22,2022-01-31 19:27:33,14th & L St NW,31283.0,10th & G St NW,31274.0,38.903658,-77.031737,38.898243,-77.026235,member
2,30DD8A84164843AD,classic_bike,2022-01-07 15:28:39,2022-01-07 15:31:01,14th & L St NW,31283.0,12th & L St NW,31251.0,38.903658,-77.031737,38.903819,-77.028400,member
3,FC67665D7682D0A6,classic_bike,2022-01-27 20:09:25,2022-01-27 20:37:02,New York Ave & Hecht Ave NE,31518.0,Nannie Helen Burroughs & Minnesota Ave NE,31704.0,38.915604,-76.983683,38.901385,-76.941877,casual
4,7854F7CC4F631A1E,classic_bike,2022-01-07 16:14:28,2022-01-07 16:16:13,Falls Church City Hall / Park Ave & Little Fal...,32608.0,Pennsylvania Ave & Park Ave,32603.0,38.885434,-77.173605,38.887403,-77.176992,member


In [5]:
#Instantiates CreateGraph class from network_tolerance_nx
network = CreateGraph()

In [6]:
df = network.preprocess_df(unprocesseddf2022)

In [7]:
# Create two graph objects for each node deletion method
# Node deletion: random failure and targeted attack
G = network.create_network(df)
G2 = G.copy()

In [8]:
print("Number of nodes:", G.number_of_nodes())
print("Number of edges:", G.number_of_edges())
density = nx.density(G)
print("Network density:", density)
triadic_closure = nx.transitivity(G)
print("Triadic closure:", triadic_closure)

print("Number of nodes:", G2.number_of_nodes())
print("Number of edges:", G2.number_of_edges())
density = nx.density(G2)
print("Network density:", density)
triadic_closure = nx.transitivity(G2)
print("Triadic closure:", triadic_closure)

Number of nodes: 709
Number of edges: 119221
Network density: 0.23750527917891834
Triadic closure: 0.6932699773435235
Number of nodes: 709
Number of edges: 119221
Network density: 0.23750527917891834
Triadic closure: 0.6932699773435235


In [9]:
nx.is_weighted(G)

True

In [10]:
# Custom measures to workaround NetworkX errors and non-existent built-in functions 
def max_diameter(G):
    max_diameter = 0
    scc = nx.strongly_connected_components(G)
    for component in scc:
        subgraph = G.subgraph(component)
        if nx.is_strongly_connected(subgraph):
            eccentricity = nx.eccentricity(subgraph)
            diameter = nx.diameter(subgraph, e=eccentricity)
            if diameter > max_diameter:
                max_diameter = diameter
        else:
            wcc = nx.weakly_connected_components(subgraph)
            diameters = []
            for wcc_component in wcc:
                wcc_subgraph = subgraph.subgraph(wcc_component)
                eccentricity = nx.eccentricity(wcc_subgraph)
                diameter = nx.diameter(wcc_subgraph, e=eccentricity)
                diameters.append(diameter)
            if diameters:
                component_max_diameter = max(diameters)
                if component_max_diameter > max_diameter:
                    max_diameter = component_max_diameter
    return max_diameter

def max_degree(G):
    degrees = dict(G.degree())
    max_degree = max(degrees.values())

    return max_degree


In [11]:
f=0.10
steps=12

#Measures to monitor as we delete nodes. Measures MUST BE a list or tuple
#Custom measures MUST be a dictionary
graph_measures = ['average_shortest_path_length']
custom_funcs = {'diameter': max_diameter, 'maxdegree': max_degree}

In [12]:
#Instantiates GraphTolerance class from network_tolerance_nx
tolerance1 = GraphTolerance(G)
tolerance2 = GraphTolerance(G2)


In [13]:
#Implement 10% random node deletion
#Steps indicate minimum datapoints to be recorded as deleted nodes approach 10%
rf_df_1 = tolerance1.random_fail(f=0.10, steps= 12, 
                         graph_measures=graph_measures,
                         custom_measures=custom_funcs)
rf_df_1

In [ ]:
results_df_2 = tolerance2.target_attack(f=0.10, steps= 12, 
                         graph_measures=graph_measures,
                         custom_measures=custom_funcs)
results_df_2

,f,f_count,average_shortest_path_length,diameter,maxdegree
0,0.007052,5,2.015764,6,779
1,0.014104,10,2.027419,6,746
2,0.021157,15,2.040365,6,712
3,0.028209,20,2.050503,6,687
4,0.035261,25,2.059314,6,673
5,0.042313,30,2.067875,6,660
6,0.049365,35,2.077921,6,641
7,0.056417,40,2.085972,6,623
8,0.063470,45,2.095702,6,607
9,0.070522,50,2.106498,6,594


In [ ]:
nx.is_weakly_connected(G2)

NameError: name 'nx' is not defined